# step2 part c : fill time gap using stata 
update: June 25, 2020 

## input: from pipeline - closerate with sentiment scores and ratings 

In [1]:
pwd

C:\Users\Herbie Huang\Google Drive\Current_SolarResearch\working_june2020\solar_project\1_code


In [3]:
import delimited "../2_pipeline/closerate_withratingcounts_sent_score.csv" ,clear


(15 vars, 6,411 obs)


In [4]:
su


    Variable |        Obs        Mean    Std. Dev.       Min        Max
-------------+---------------------------------------------------------
          v1 |      6,411        3205    1850.841          0       6410
           i |      6,411        3205    1850.841          0       6410
        five |      6,411    3.846826    5.965941          0         47
        four |      6,411    .1481828     .558752          0          6
       three |      6,411    .0249571    .1618954          0          2
-------------+---------------------------------------------------------
         two |      6,411    .0210576    .1758231          0          3
         one |      6,411    .0642645    .3566885          0          5
avg_sent_s~e |      4,393     .685525    .1616075     -.8268      .9576
    unnamed0 |      6,411        3205    1850.841          0       6410
installer_id |      6,411    18437.62    6381.602        108      23027
-------------+-------------------------------------------------

## Prepare for a panel data structure ( need installer_id, and time index ( year_month_count) ) 

In [5]:
gen year=int(year_month/100)
gen month=year_month-100*year
gen year_month_count=12*(year-2013)+month
label variable year_month_count `""month since 2013""'



##  tsfill fills in the time gap

In [6]:
xtset installer_id year_month_count
tsfill



       panel variable:  installer_id (unbalanced)
        time variable:  year_month_~t, -1 to 64, but with gaps
                delta:  1 unit



In [7]:
br

## carryforward(an external stata command) will carryforward the value of these specified variable 

In [8]:
keep installer_id year_month_count five four three two one quotes_count winning_quotes_count avg_sent_score
bysort installer_id: carryforward five four three two one avg_sent_score  , replace
gen reviews_ct=(five+four+three+two+one)
gen temp_numerator=5*five+4*four+3*three+2*two+one
gen avg=temp_numerator/reviews_ct
drop temp_numerator




five:  (2,244 real changes made)
four:  (2,244 real changes made)
three:  (2,244 real changes made)
two:  (2,244 real changes made)
one:  (2,244 real changes made)
avg_sent_score:  (874 real changes made)



(3,388 missing values generated)



In [9]:
duplicates report


Duplicates in terms of all variables

--------------------------------------
   copies | observations       surplus
----------+---------------------------
        1 |         8655             0
--------------------------------------


In [11]:
su


    Variable |        Obs        Mean    Std. Dev.       Min        Max
-------------+---------------------------------------------------------
        five |      8,655    3.191912    5.474295          0         47
        four |      8,655    .1192374    .4945914          0          6
       three |      8,655     .021606    .1500942          0          2
         two |      8,655    .0157135      .15197          0          3
         one |      8,655    .0523397    .3176661          0          5
-------------+---------------------------------------------------------
avg_sent_s~e |      5,267    .6837752      .16615     -.8268      .9576
installer_id |      8,655    18033.26    6656.861        108      23027
quotes_count |      6,411    37.99189    91.81771          1       1417
winning_qu~t |      6,411    .6229917    1.636259          0         43
year_month~t |      8,655    43.11623    14.66923         -1         64
-------------+-------------------------------------------------

## output: to pipeline: es_monthly_ind.dta 

In [10]:
save "../2_pipeline/es_monthly_ind.dta",replace

(note: file ../2_pipeline/es_monthly_ind.dta not found)
file ../2_pipeline/es_monthly_ind.dta saved
